# Introduction
General housekeeping for running the notebooks and analysis pipeline.

In [11]:
import pickle as pkl
import scanpy as sc
import matplotlib as mpl

In [4]:
mountpoint = '/data/clue/'

# Create Condition Colors

I use a `pkl` called `cond_colors.pkl` in various notebooks throughout the processing. Here is where I create it.

In [19]:
cond_colors = dict()
cond_colors['hex'] = dict(zip(['0', 'A', 'B', 'C', 'G', 'P' ,'R'], sc.pl.palettes.default_20))
cond_colors['rgb'] = dict(zip(['0', 'A', 'B', 'C', 'G', 'P' ,'R'], list(map(mpl.colors.hex2color, sc.pl.palettes.default_20))))

In [20]:
# with open(mountpoint + 'cond_colors.pkl', 'wb') as file:
#     pkl.dump(cond_colors, file)

In [21]:
with open(mountpoint + 'cond_colors.pkl', 'rb') as file:
    cond_colors = pkl.load(file)